In [43]:
import os
import sys
module_path = os.path.abspath(os.path.join(r'D:\gitClones\nteract_models\optimize\scripts'))
if module_path not in sys.path:
    sys.path.append(module_path)
from scipy.optimize import minimize
from scripts.util import q_ge, tailwater_tdg
import pandas as pd
import papermill as pm

In [38]:
#Params
train ={}
test = {}
maxiter = 100
x0 = [1,.5,1]
min_fb_tdg = '-inf'

In [18]:
min_fb_tdg = float(min_fb_tdg)
#put data in dataframe
train = pd.DataFrame(train)
train['index'] = pd.to_datetime(train['index'])
train.set_index('index', inplace = True)

test = pd.DataFrame(test)
test['index'] = pd.to_datetime(test['index'])
test.set_index('index', inplace = True)

#separate only data with spill.  A separate process will be optimized with no spill conditions
spill_train = train[(train['q_s']>0) & (train['tdg_f']> min_fb_tdg)]

spill_test = test[(test['q_s']>0) & (test['tdg_f']> min_fb_tdg)]

In [20]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(y, y_hat):
    return sqrt(mean_squared_error(y,y_hat))

#See scripts/utils.py for explanation of spillway_powerhouse_flows, tailwater, and powerhouse_entrainment or reference

#Reference:
#Stewart, Kevin M., Adam Witt, and Boualem Hadjerioua. 
#"Total dissolved gas prediction and optimization in RIVERWARE." 
#Prepared for US Department of Energy Wind and Water Program by 
#Oakridge National Laboratory, Oak Ridge, TN (2015).
#https://info.ornl.gov/sites/publications/files/Pub59285.pdf


#The objective function is what the optimization will try to minimize with x, the b1, b2, and b3 coefficients.
def objective(x):
    b1,b2,b3 = x
    y_hat = spill_train.apply(lambda r:tailwater_tdg(r['q_s'],r['q_p'],b1,b2,b3, r['h_t'],r['temp_water'],r['p_atm'],r['tdg_f']), axis = 1)
    y = spill['tdg_tw']
    return rmse(y, y_hat)

#constraints and bounds for optimization model.  See reference for more information
def constraint_q_ge(x):
    b1,b2,b3=x
    power_flow = spill_train.apply(lambda x:q_ge(x['q_p'],x['q_s'],b1,b3), axis = 1)
    const = power_flow<0
    return -const.sum()

constraints = [{'type':'ineq', 'fun':constraint_q_ge}]          
bounds = [(None,None),(.00001,None),(None,None)]

In [ ]:
#optimization process
sol = minimize(objective, x0, method = 'SLSQP', constraints = constraints, bounds = bounds,options={'disp':True, 'maxiter':maxiter})
x = sol.x

In [ ]:
#for cross validattion
y_hat = spill_test.apply(lambda r:tailwater_tdg(r['q_s'],r['q_p'],b1,b2,b3, r['h_t'],r['temp_water'],r['p_atm'],r['tdg_f']), axis = 1)
y = spill_test['tdg_tw']

#save to notebook
pm.record('x',x)
pm.record('y',y)
pm.record('y_hat',y_hat)